In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
import nltk
from nltk.corpus import stopwords
stpwords =  set(stopwords.words('english'))

In [3]:
import spacy
nlp = spacy.load("en_core_web_lg",disable=["ner"])

In [4]:
def get_correct_option_only(text,corr_option):
#     text = "Which describes a material that is not a food? (A) It stores energy but not nutrients. (B) It does not store energy or nutrients. (C) It stores energy and nutrients. (D) It does not store energy but stores nutrients."
    text_question = text.split("(A)")[0].strip()
    try:
        correct_option = text.split("("+corr_option+")")[1].split("(")[0]
    except:
        print(text)
#     print(text_question,correct_option)
    return text_question,correct_option

In [5]:
def get_explanations(exps):
#     exps = 'eccc-ccef-2b84-af94|CENTRAL dd64-2412-f733-7659|LEXGLUE 0456-0148-2518-a48e|LEXGLUE 004d-1843-0f4a-8ad4|LEXGLUE b964-cdbd-a226-4027|CENTRAL 1ab8-7900-02c0-a2d6|CENTRAL d7c1-cdfd-2de5-6b7d|CENTRAL 1015-4572-2293-d185|LEXGLUE 0d3a-8739-9f42-5e1c|GROUNDING f6bf-c88d-502b-826b|CENTRAL 10ce-c060-90b9-b748|GROUNDING ff68-4699-0671-f8b7|LEXGLUE 7a74-426d-ec77-f296|LEXGLUE	'
    try:
        exps = exps.split(" ")
    except:
#         print(exps)
        pass
    
    out_list = [ x.split("|")[0].strip() for x in exps ]
#     print(out_list)
    return out_list

def get_roles(exps):
#     exps = 'eccc-ccef-2b84-af94|CENTRAL dd64-2412-f733-7659|LEXGLUE 0456-0148-2518-a48e|LEXGLUE 004d-1843-0f4a-8ad4|LEXGLUE b964-cdbd-a226-4027|CENTRAL 1ab8-7900-02c0-a2d6|CENTRAL d7c1-cdfd-2de5-6b7d|CENTRAL 1015-4572-2293-d185|LEXGLUE 0d3a-8739-9f42-5e1c|GROUNDING f6bf-c88d-502b-826b|CENTRAL 10ce-c060-90b9-b748|GROUNDING ff68-4699-0671-f8b7|LEXGLUE 7a74-426d-ec77-f296|LEXGLUE	'
    if "|" in exps:
        try:
            exps = exps.split(" ")
        except:
#             print(exps)
            pass

        out_list = [ x.split("|")[1].strip() for x in exps ]
    #     print(out_list)
        return out_list
    else:
        try:
            exps = exps.split(" ")
        except:
#             print(exps)
            pass
        return ["CENTRAL"]*len(exps)

def get_text_explanations(exps):
    outlist = []
    for x in exps:
        try: 
            text = explanation_map[x]
            outlist.append(text)
        except:
            print("Not Found:",x)
    return outlist

# get_text_explanations(get_explanations("a"))

In [6]:
train = pd.read_csv("../train.csv",)
dev = pd.read_csv("../dev.csv")
test = pd.read_csv("../test.csv")

In [7]:
def test_nan(df):
    print("Total Length:",len(df))
    print(df[df.isna().any(axis=1)].count())

In [8]:
test_nan(train),test_nan(dev),test_nan(test)

Total Length: 1190
Unnamed: 0            203
questionID            203
originalQuestionID    203
Question              203
AnswerKey             203
explanation             0
dtype: int64
Total Length: 264
Unnamed: 0            38
questionID            38
originalQuestionID    38
Question              38
AnswerKey             38
explanation            0
dtype: int64
Total Length: 1247
Unnamed: 0            1247
questionID            1247
originalQuestionID    1247
Question              1247
AnswerKey             1247
explanation              0
dtype: int64


(None, None, None)

In [9]:
# Note : Train and Dev have few datapoints with no explanations. Ignore them

In [10]:
explanations = pd.read_csv("../explanations.csv")

In [11]:
len(explanations)

4950

In [12]:
explanations.head()

,Unnamed: 0,uid,text
0,0,14a5-1229-3653-681c,a landslide is when water; gravity rapidly mov...
1,1,12d9-60df-23de-f00f,decomposition is when a decomposer recycles; r...
2,2,e1fb-b791-3945-02db,deposition means to move an object from one lo...
3,3,244b-03a0-929e-0095,eating;digestion is when an organism takes in ...
4,4,22ce-038b-ff4f-c5ba,electrical conduction is when metals; electric...


In [13]:
explanation_map = {}
for index,row in tqdm(explanations.iterrows()):
    explanation_map[row['uid']] = row['text']

4950it [00:01, 4437.98it/s]


In [ ]:
explanation_map['12d9-60df-23de-f00f']

In [15]:
docmap= {}

In [17]:
from random import shuffle
from itertools import cycle, islice
import operator

def strip_punct(s):
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in s if ch not in exclude)
    return s

def get_doc(docs):
    if docs in docmap:
        return docmap[docs]
    docmap[docs] = nlp(docs)
    return docmap[docs]

def get_other_facts(factlist,l=50):
    wronglist = []
    explist = [x for x in explanation_map.items()]
    shuffle(explist)
    scored_wfact_map = {}
    for uid,wfact in explist:
        present = False
        wdoc = get_doc(wfact)
        score = -1
        for fact in factlist:
            if fact == wfact:
                present = True
                break
            fdoc =get_doc(fact)
            score = max(score,fdoc.similarity(wdoc))
        if not present:
            scored_wfact_map[uid]=score
    
    sorted_x = list(sorted(scored_wfact_map.items(), key=operator.itemgetter(1),reverse=True))[0:l]
    for tup in sorted_x:
        if tup[0] not in wronglist:
            wronglist.append([tup[0],explanation_map[tup[0]]])
    return wronglist

def get_other_fact_v2(fact,l=20):
    wronglist = []
    explist = [x for x in explanation_map.items()]
    shuffle(explist)
    scored_wfact_map = {}
    score=-1
    for uid,wfact in explist:
        present = False
        wdoc = get_doc(wfact)
        score = max(score,fdoc.similarity(wdoc))
        if not present:
            scored_wfact_map[uid]=score
            
    sorted_x = list(sorted(scored_wfact_map.items(), key=operator.itemgetter(1),reverse=True))[0:l]
    for tup in sorted_x:
        if [tup[0],explanation_map[tup[0]]] not in wronglist:
            wronglist.append([tup[0],explanation_map[tup[0]]])
    return wronglist
        
def get_same_facts(factlist,l=50):
    return list(islice(cycle(factlist), l))

get_other_facts(['earthworms create tunnels in soil',
 'to create means to make',
 'a tunnel is a kind of hole',
 'soil is a part of the ground outside',
 'tunnels in soil loosen that soil',
 'the looseness of soil increases the amount of oxygen in that soil',
 'plants absorb nutrients; water; oxygen from soil into themselves through their roots',
 'taking in something means receiving; absorbing; getting something',
 'a tree is a kind of plant',
 'an earthworm is a kind of animal',
 'an animal is a kind of organism',
 'to make something easier means to help',
 'getting something increases the amount of that something'])
            

[['bbaf-fd47-e244-3163',
  'losing something decreases the amount of that something'],
 ['ea5f-42f2-93e0-9520', 'a leaf is a kind of plant'],
 ['9cf1-bc5a-d350-5541', 'a lemon tree is a kind of plant'],
 ['e337-f5c8-75aa-81b6', 'a flower is a kind of plant'],
 ['9a85-698e-680e-5033', 'a fern is a kind of plant'],
 ['311e-8d88-cde2-860d', 'a bush is a kind of plant'],
 ['c23b-f665-cd04-dafd', 'to produce means to create'],
 ['aadb-cb93-3373-eefa', 'a cactus is a kind of plant'],
 ['84bb-0231-2480-1c8b',
  'destroying something decreases the amount of that something'],
 ['da68-3173-2681-d90e',
  'a source of something increases the amount of that something'],
 ['1653-a7fe-e9be-42ca',
  'preventing something decreases the amount of that something'],
 ['56bd-8c40-3dae-d3e9', 'an insect is a kind of animal'],
 ['daad-f76d-b43d-ce31', 'to form means to create'],
 ['679c-fab2-a001-db1e',
  'to be used  for something means to help to do that something'],
 ['00a7-56fc-3c22-e6f2', 'an apple tree

In [18]:
dev.head()

,Unnamed: 0,questionID,originalQuestionID,Question,AnswerKey,explanation
0,0,VASoL_2008_3_26,26,Which of these is MOST flexible? (A) Broom han...,C,98b6-2bc9-0f82-84f9|CENTRAL ac40-d9c4-86a2-f4b...
1,1,Mercury_SC_415491,415491,Earth orbits the Sun once a year. About how ma...,C,1a29-2268-eeb7-edba|CENTRAL 1a4b-82df-0613-526...
2,2,VASoL_2007_3_33,33,Which of these causes the MOST evaporation of ...,B,cb74-f946-eef2-3ca5|CENTRAL a0b2-a45f-01e1-4bf...
3,3,Mercury_SC_415772,415772,"If you bounce a rubber ball on the floor, it g...",B,9773-5df7-84db-74b3|CENTRAL 5650-b213-46ea-d9d...
4,4,Mercury_SC_415366,415366,Trees need oxygen. Roots close to the surface ...,B,eccc-ccef-2b84-af94|CENTRAL dd64-2412-f733-765...


In [123]:
# Prepare a Ranking Classification Dataset

def create_2_class_dataset(df,fname):
    lengthmap= {}
    with open("mod_data3/"+fname,"w+") as outfd:
        df_t = df.dropna()
        for index,row in tqdm(df_t.iterrows()):
            qid = row['questionID']
            cor_opt = row['AnswerKey']
            explanations = get_explanations(row['explanation'])
            exp_texts = get_text_explanations(explanations)
            ques,ans = get_correct_option_only(row['Question'],cor_opt)
            wrong_facts = get_other_facts(exp_texts)
            hyp = ques + " " + ans #mod_data1 and 2- 2 was better
#             hyp = row['Question']  # mod_data3
            for uid,fact in get_same_facts(list(zip(explanations,exp_texts))):
                idx = qid+":"+uid
                outfd.write("%s\t%s\t%s\t%d\n"%(idx,hyp,fact,1))
            for uid,fact in wrong_facts:
                idx = qid+":"+uid
                outfd.write("%s\t%s\t%s\t%d\n"%(idx,hyp,fact,0))
            if len(explanations) not in lengthmap:
                lengthmap[len(explanations)]=0
            lengthmap[len(explanations)]+=1
    print(lengthmap)

In [133]:
def create_n_class_dataset(df,fname):
    lengthmap= {}
    with open("mod_datan/"+fname,"w+") as outfd:
        df_t = df.dropna()
        for index,row in tqdm(df_t.iterrows()):
            qid = row['questionID']
            cor_opt = row['AnswerKey']
            explanations = get_explanations(row['explanation'])
            exp_texts = get_text_explanations(explanations)
            ques,ans = get_correct_option_only(row['Question'],cor_opt)
            wrong_facts = get_other_facts(exp_texts)
            hyp = ques + " " + ans # mod_data1 and 2
#             hyp = row['Question']  # mod_data3
# Initial length
            for uid,fact in get_same_facts(list(zip(explanations,exp_texts))):
                idx = qid+":"+uid
                outfd.write("%s\t%s\t%s\t%d\n"%(idx,hyp,fact,1))
            for uid,fact in wrong_facts:
                idx = qid+":"+uid
                outfd.write("%s\t%s\t%s\t%d\n"%(idx,hyp,fact,0))
            if len(explanations) not in lengthmap:
                lengthmap[len(explanations)]=0
# Consequent length                
            for index,exp in enumerate(exp_texts):
                hyp = hyp + " . " + exp
                try:
                    wrong_facts = get_other_facts(exp_texts[index+1:])
                    for uid,fact in get_same_facts(list(zip(explanations,exp_texts[index+1:]))):
                        idx = qid+":"+uid+":"+str(index)
                        outfd.write("%s\t%s\t%s\t%d\n"%(idx,hyp,fact,1))
                    for uid,fact in wrong_facts:
                        idx = qid+":"+uid+":"+str(index)
                        outfd.write("%s\t%s\t%s\t%d\n"%(idx,hyp,fact,0))
                except:
                    pass
                
            lengthmap[len(explanations)]+=1
    print(lengthmap)

In [21]:
roles_map = {
    'CENTRAL' : 1,
    'LEXGLUE' : 2,
    'GROUNDING' : 3,
    'ROLE' : 1
}

def create_n_class_multi_dataset(df,fname):
    lengthmap= {}
    with open("mod_datamulti/"+fname,"w+") as outfd:
        df_t = df.dropna()
        for index,row in tqdm(df_t.iterrows()):
            qid = row['questionID']
            cor_opt = row['AnswerKey']
            explanations = get_explanations(row['explanation'])
            exp_texts = get_text_explanations(explanations)
            roles = get_roles(row['explanation'])
#             print(roles)
            ques,ans = get_correct_option_only(row['Question'],cor_opt)
            wrong_facts = get_other_facts(exp_texts)
            hyp = ques + " " + ans # mod_data1 and 2
#             hyp = row['Question']  # mod_data3
# Initial length
            for uid,fact,role in get_same_facts(list(zip(explanations,exp_texts,roles))):
                idx = qid+":"+uid
                outfd.write("%s\t%s\t%s\t%d\n"%(idx,hyp,fact,roles_map[role]))
            for uid,fact in wrong_facts:
                idx = qid+":"+uid
                outfd.write("%s\t%s\t%s\t%d\n"%(idx,hyp,fact,0))
            if len(explanations) not in lengthmap:
                lengthmap[len(explanations)]=0
# Consequent length                
            for index,exp in enumerate(exp_texts):
                hyp = hyp + " . " + exp
                try:
                    wrong_facts = get_other_facts(exp_texts[index+1:])
                    for uid,fact,role in get_same_facts(list(zip(explanations,exp_texts[index+1:],roles[index+1:]))):
                        idx = qid+":"+uid+":"+str(index)
                        outfd.write("%s\t%s\t%s\t%d\n"%(idx,hyp,fact,roles_map[role]))
                    for uid,fact in wrong_facts:
                        idx = qid+":"+uid+":"+str(index)
                        outfd.write("%s\t%s\t%s\t%d\n"%(idx,hyp,fact,0))
                except:
                    pass
                
            lengthmap[len(explanations)]+=1
    print(lengthmap)

In [125]:
def create_2_class_dataset_test(df,fname):
    with open("mod_data3/"+fname,"w+") as outfd:
        for index,row in tqdm(df.iterrows()):
            qid = row['questionID']
            cor_opt = row['AnswerKey']
#             ques,ans = get_correct_option_only(row['Question'],cor_opt)
#             hyp = ques + " " + ans
            hyp = row['Question']
            for uid,fact in explanation_map.items():
                idx = qid+":"+uid
                outfd.write("%s\t%s\t%s\t%d\n"%(idx,hyp,fact,0))

In [126]:
create_2_class_dataset(dev,"dev.tsv")


0it [00:00, ?it/s]
1it [00:00,  8.47it/s]
3it [00:00,  8.28it/s]
4it [00:00,  8.50it/s]
5it [00:00,  5.29it/s]
6it [00:00,  5.61it/s]
7it [00:01,  4.92it/s]
8it [00:01,  5.02it/s]
9it [00:01,  5.44it/s]
10it [00:01,  5.83it/s]
12it [00:01,  7.22it/s]
13it [00:02,  6.49it/s]
14it [00:02,  5.76it/s]
15it [00:02,  6.33it/s]
17it [00:02,  5.91it/s]
19it [00:03,  6.31it/s]
20it [00:03,  6.04it/s]
22it [00:03,  6.51it/s]
23it [00:03,  6.18it/s]
25it [00:03,  7.35it/s]
26it [00:03,  7.54it/s]
27it [00:04,  6.56it/s]
28it [00:04,  6.03it/s]
29it [00:04,  6.14it/s]
30it [00:04,  6.78it/s]
31it [00:04,  5.52it/s]
32it [00:05,  4.85it/s]
33it [00:05,  4.44it/s]
34it [00:05,  4.53it/s]
35it [00:05,  4.72it/s]
36it [00:05,  4.96it/s]
37it [00:06,  5.13it/s]
38it [00:06,  4.86it/s]
40it [00:06,  4.98it/s]
42it [00:06,  6.26it/s]
43it [00:07,  5.95it/s]
45it [00:07,  7.11it/s]
46it [00:07,  5.22it/s]
48it [00:07,  5.83it/s]
49it [00:08,  4.85it/s]
50it [00:08,  5.43it/s]
51it [00:08,  5.78it/s]
52it

{4: 25, 2: 18, 6: 29, 3: 34, 13: 2, 9: 14, 7: 17, 5: 24, 1: 12, 8: 20, 11: 7, 10: 7, 12: 6, 16: 4, 18: 2, 17: 2, 22: 1, 14: 1, 15: 1}


In [127]:
create_2_class_dataset(train,"train.tsv")


0it [00:00, ?it/s]
1it [00:00,  3.80it/s]
2it [00:00,  3.99it/s]
3it [00:00,  3.24it/s]
4it [00:01,  3.82it/s]
5it [00:01,  4.06it/s]
6it [00:01,  4.60it/s]
7it [00:01,  4.58it/s]
8it [00:01,  4.67it/s]
9it [00:02,  4.32it/s]
10it [00:02,  4.49it/s]
12it [00:02,  5.19it/s]
13it [00:02,  4.42it/s]
15it [00:03,  5.06it/s]
17it [00:03,  5.50it/s]
18it [00:03,  6.19it/s]
19it [00:03,  5.20it/s]
20it [00:04,  4.92it/s]
21it [00:04,  4.82it/s]
23it [00:04,  5.78it/s]
24it [00:04,  5.85it/s]
25it [00:04,  6.23it/s]
26it [00:04,  5.84it/s]
27it [00:05,  6.24it/s]
28it [00:05,  6.84it/s]
29it [00:05,  5.08it/s]
30it [00:05,  5.43it/s]
31it [00:05,  5.53it/s]
32it [00:06,  5.63it/s]
34it [00:06,  6.97it/s]
35it [00:06,  6.46it/s]
36it [00:06,  6.97it/s]
38it [00:06,  7.81it/s]
39it [00:06,  6.86it/s]
41it [00:07,  7.63it/s]
42it [00:07,  7.49it/s]
43it [00:07,  6.76it/s]
45it [00:07,  6.99it/s]
46it [00:07,  7.12it/s]
47it [00:07,  6.45it/s]
48it [00:08,  5.14it/s]
49it [00:08,  5.53it/s]
50it 

Not Found: f3bd-0290-ae95-8e0a



66it [00:11,  5.99it/s]
67it [00:11,  5.53it/s]
68it [00:11,  5.74it/s]
70it [00:11,  7.14it/s]
71it [00:11,  6.96it/s]
72it [00:11,  7.56it/s]
73it [00:12,  5.15it/s]
74it [00:12,  5.25it/s]
76it [00:12,  5.77it/s]
77it [00:12,  6.60it/s]
78it [00:13,  3.75it/s]
80it [00:13,  4.80it/s]
81it [00:13,  5.32it/s]
82it [00:13,  6.06it/s]
83it [00:13,  5.86it/s]
84it [00:14,  5.92it/s]
85it [00:14,  5.80it/s]
86it [00:14,  5.74it/s]
87it [00:14,  5.76it/s]
89it [00:14,  6.64it/s]
90it [00:15,  5.27it/s]
91it [00:15,  5.38it/s]
92it [00:15,  5.73it/s]
93it [00:15,  5.73it/s]
94it [00:15,  6.54it/s]
95it [00:16,  4.89it/s]
97it [00:16,  5.86it/s]
99it [00:16,  6.24it/s]
100it [00:16,  6.83it/s]
101it [00:16,  5.71it/s]
102it [00:17,  6.04it/s]
104it [00:17,  6.29it/s]
106it [00:17,  5.72it/s]
107it [00:17,  5.92it/s]
108it [00:18,  6.07it/s]
109it [00:18,  6.33it/s]
110it [00:18,  6.77it/s]
112it [00:18,  7.31it/s]
113it [00:18,  7.57it/s]
114it [00:18,  5.22it/s]
115it [00:19,  4.13it/s]
11

Not Found: 5124-b03e-ff20-b4d6



624it [01:49,  5.41it/s]
625it [01:49,  4.60it/s]
626it [01:50,  3.69it/s]
627it [01:50,  4.45it/s]
628it [01:50,  3.89it/s]
629it [01:50,  4.67it/s]
630it [01:51,  4.51it/s]
632it [01:51,  5.35it/s]
633it [01:51,  4.63it/s]
634it [01:51,  5.01it/s]
636it [01:52,  4.76it/s]
637it [01:52,  4.72it/s]
638it [01:52,  4.98it/s]
640it [01:52,  5.44it/s]
641it [01:53,  5.20it/s]
642it [01:53,  5.70it/s]
643it [01:53,  6.46it/s]
645it [01:53,  7.22it/s]
646it [01:53,  6.63it/s]
648it [01:54,  6.36it/s]
649it [01:54,  6.91it/s]
650it [01:54,  6.59it/s]
651it [01:54,  6.64it/s]
652it [01:54,  5.56it/s]
653it [01:55,  4.90it/s]
654it [01:55,  3.78it/s]
655it [01:55,  4.03it/s]
656it [01:55,  4.56it/s]
657it [01:56,  4.42it/s]
658it [01:56,  4.11it/s]
659it [01:56,  4.98it/s]
660it [01:56,  5.35it/s]
661it [01:56,  5.91it/s]
662it [01:56,  6.08it/s]
663it [01:57,  5.89it/s]
664it [01:57,  5.67it/s]
665it [01:57,  6.22it/s]
667it [01:57,  6.87it/s]
668it [01:57,  6.62it/s]
669it [01:57,  5.57it/s]

Not Found: 5124-b03e-ff20-b4d6



833it [02:25,  5.84it/s]
834it [02:26,  4.24it/s]
836it [02:26,  4.91it/s]
838it [02:26,  6.34it/s]
840it [02:26,  6.02it/s]
842it [02:27,  7.35it/s]
844it [02:27,  7.85it/s]
846it [02:27,  7.98it/s]
847it [02:27,  6.17it/s]
848it [02:27,  6.19it/s]
850it [02:28,  6.78it/s]
851it [02:28,  7.01it/s]
852it [02:28,  7.29it/s]
853it [02:28,  7.58it/s]
854it [02:28,  8.06it/s]
856it [02:28,  7.16it/s]
857it [02:29,  7.59it/s]
858it [02:29,  7.93it/s]
860it [02:29,  8.14it/s]
861it [02:29,  7.58it/s]
862it [02:29,  6.09it/s]
864it [02:29,  7.26it/s]
865it [02:30,  6.77it/s]
867it [02:30,  7.31it/s]
869it [02:30,  7.83it/s]
870it [02:30,  8.26it/s]
871it [02:30,  6.55it/s]
872it [02:31,  6.88it/s]
873it [02:31,  6.57it/s]
874it [02:31,  6.47it/s]
875it [02:31,  6.93it/s]
877it [02:31,  7.81it/s]
879it [02:32,  6.95it/s]
880it [02:32,  7.04it/s]
881it [02:32,  6.10it/s]
883it [02:32,  7.37it/s]
884it [02:32,  7.82it/s]
885it [02:32,  6.71it/s]
886it [02:32,  7.10it/s]
888it [02:33,  7.62it/s]

{11: 25, 8: 59, 18: 4, 6: 110, 9: 52, 10: 49, 7: 75, 3: 139, 5: 105, 12: 31, 2: 95, 4: 124, 1: 49, 17: 6, 14: 10, 21: 6, 13: 25, 16: 9, 15: 11, 20: 2, 19: 1}


In [135]:
create_n_class_dataset(train,"train.tsv")


0it [00:00, ?it/s]
1it [00:01,  1.58s/it]
2it [00:02,  1.38s/it]
3it [00:06,  2.11s/it]
4it [00:06,  1.65s/it]
5it [00:07,  1.49s/it]
6it [00:08,  1.21s/it]
7it [00:09,  1.19s/it]
8it [00:10,  1.11s/it]
9it [00:12,  1.19s/it]
10it [00:12,  1.05s/it]
11it [00:12,  1.27it/s]
12it [00:13,  1.45it/s]
13it [00:15,  1.04s/it]
14it [00:15,  1.27it/s]
15it [00:16,  1.31it/s]
16it [00:16,  1.74it/s]
17it [00:17,  1.35it/s]
18it [00:17,  1.73it/s]
19it [00:19,  1.16it/s]
20it [00:20,  1.03it/s]
21it [00:21,  1.07s/it]
22it [00:21,  1.21it/s]
23it [00:22,  1.52it/s]
24it [00:22,  1.46it/s]
25it [00:23,  1.66it/s]
26it [00:24,  1.51it/s]
27it [00:24,  1.85it/s]
28it [00:24,  1.99it/s]
29it [00:27,  1.11s/it]
30it [00:28,  1.02s/it]
31it [00:28,  1.06it/s]
32it [00:29,  1.21it/s]
33it [00:29,  1.60it/s]
35it [00:30,  1.78it/s]
36it [00:30,  1.97it/s]
38it [00:31,  2.23it/s]
39it [00:32,  1.62it/s]
40it [00:32,  2.10it/s]
41it [00:32,  2.43it/s]
42it [00:33,  2.21it/s]
43it [00:34,  1.79it/s]
44it 

Not Found: f3bd-0290-ae95-8e0a



66it [00:56,  1.18it/s]
67it [00:58,  1.04s/it]
68it [00:58,  1.01it/s]
69it [00:59,  1.27it/s]
71it [01:00,  1.49it/s]
72it [01:00,  1.79it/s]
73it [01:02,  1.15s/it]
74it [01:03,  1.10s/it]
75it [01:04,  1.23it/s]
76it [01:05,  1.16it/s]
77it [01:05,  1.47it/s]
78it [01:12,  2.64s/it]
79it [01:12,  1.89s/it]
80it [01:12,  1.36s/it]
81it [01:13,  1.07s/it]
82it [01:13,  1.21it/s]
83it [01:14,  1.23it/s]
84it [01:14,  1.27it/s]
85it [01:15,  1.17it/s]
86it [01:16,  1.20it/s]
87it [01:17,  1.16it/s]
88it [01:17,  1.52it/s]
89it [01:18,  1.43it/s]
90it [01:20,  1.08s/it]
91it [01:21,  1.02it/s]
92it [01:21,  1.31it/s]
93it [01:22,  1.43it/s]
94it [01:22,  1.74it/s]
95it [01:25,  1.29s/it]
96it [01:25,  1.06it/s]
97it [01:25,  1.35it/s]
98it [01:26,  1.67it/s]
99it [01:26,  1.54it/s]
100it [01:27,  1.87it/s]
101it [01:28,  1.33it/s]
102it [01:28,  1.56it/s]
103it [01:28,  2.03it/s]
104it [01:30,  1.26it/s]
105it [01:30,  1.58it/s]
106it [01:33,  1.42s/it]
107it [01:34,  1.16s/it]
108it [

Not Found: 5124-b03e-ff20-b4d6



623it [09:47,  1.26it/s]
624it [09:47,  1.39it/s]
625it [09:49,  1.03it/s]
626it [09:52,  1.52s/it]
627it [09:52,  1.15s/it]
628it [09:54,  1.38s/it]
629it [09:54,  1.07s/it]
630it [09:55,  1.04it/s]
631it [09:55,  1.36it/s]
632it [09:56,  1.55it/s]
633it [09:57,  1.08it/s]
634it [09:58,  1.22it/s]
636it [10:01,  1.01it/s]
637it [10:01,  1.01it/s]
638it [10:02,  1.16it/s]
639it [10:02,  1.52it/s]
640it [10:03,  1.49it/s]
641it [10:04,  1.47it/s]
642it [10:04,  1.77it/s]
643it [10:04,  2.20it/s]
644it [10:04,  2.81it/s]
645it [10:05,  2.62it/s]
646it [10:05,  2.12it/s]
648it [10:08,  1.54it/s]
649it [10:08,  1.84it/s]
650it [10:08,  1.81it/s]
651it [10:09,  2.07it/s]
652it [10:10,  1.24it/s]
653it [10:11,  1.13it/s]
654it [10:14,  1.44s/it]
655it [10:15,  1.28s/it]
656it [10:16,  1.06s/it]
657it [10:17,  1.07s/it]
658it [10:18,  1.28s/it]
659it [10:19,  1.05it/s]
660it [10:19,  1.33it/s]
661it [10:19,  1.59it/s]
662it [10:20,  1.76it/s]
663it [10:20,  1.94it/s]
664it [10:21,  1.74it/s]

Not Found: 5124-b03e-ff20-b4d6



832it [12:34,  1.55it/s]
833it [12:34,  1.60it/s]
834it [12:38,  1.47s/it]
835it [12:38,  1.09s/it]
836it [12:38,  1.06it/s]
838it [12:39,  1.48it/s]
839it [12:39,  1.78it/s]
840it [12:40,  1.23it/s]
841it [12:40,  1.57it/s]
843it [12:41,  2.02it/s]
844it [12:41,  2.27it/s]
845it [12:41,  2.72it/s]
846it [12:42,  2.29it/s]
847it [12:43,  1.87it/s]
848it [12:43,  1.83it/s]
849it [12:43,  2.27it/s]
850it [12:44,  2.04it/s]
851it [12:44,  2.12it/s]
852it [12:45,  2.18it/s]
853it [12:45,  2.40it/s]
854it [12:45,  2.85it/s]
855it [12:46,  3.51it/s]
856it [12:47,  1.33it/s]
857it [12:48,  1.61it/s]
858it [12:48,  1.92it/s]
859it [12:48,  2.51it/s]
860it [12:49,  2.19it/s]
861it [12:49,  2.29it/s]
862it [12:50,  1.60it/s]
864it [12:50,  2.09it/s]
865it [12:51,  2.02it/s]
867it [12:52,  2.16it/s]
869it [12:52,  2.28it/s]
870it [12:53,  2.48it/s]
871it [12:54,  1.66it/s]
872it [12:54,  1.96it/s]
873it [12:55,  1.93it/s]
874it [12:55,  1.89it/s]
875it [12:56,  2.14it/s]
876it [12:56,  2.75it/s]

{11: 25, 8: 59, 18: 4, 6: 110, 9: 52, 10: 49, 7: 75, 3: 139, 5: 105, 12: 31, 2: 95, 4: 124, 1: 49, 17: 6, 14: 10, 21: 6, 13: 25, 16: 9, 15: 11, 20: 2, 19: 1}


In [136]:
create_n_class_dataset(dev,"dev.tsv")


0it [00:00, ?it/s]
1it [00:00,  2.04it/s]
2it [00:00,  2.48it/s]
3it [00:01,  1.76it/s]
4it [00:01,  2.01it/s]
5it [00:05,  1.51s/it]
6it [00:06,  1.20s/it]
7it [00:08,  1.42s/it]
8it [00:09,  1.37s/it]
9it [00:10,  1.17s/it]
10it [00:10,  1.04it/s]
12it [00:10,  1.40it/s]
13it [00:11,  1.27it/s]
14it [00:13,  1.05it/s]
15it [00:13,  1.26it/s]
16it [00:13,  1.66it/s]
17it [00:15,  1.06s/it]
18it [00:16,  1.23it/s]
19it [00:16,  1.28it/s]
20it [00:17,  1.30it/s]
21it [00:17,  1.68it/s]
22it [00:18,  1.57it/s]
23it [00:19,  1.36it/s]
25it [00:19,  1.78it/s]
26it [00:20,  1.97it/s]
27it [00:21,  1.58it/s]
28it [00:21,  1.49it/s]
29it [00:22,  1.59it/s]
30it [00:22,  1.95it/s]
31it [00:24,  1.28it/s]
32it [00:25,  1.02it/s]
33it [00:27,  1.20s/it]
34it [00:28,  1.19s/it]
35it [00:29,  1.11s/it]
36it [00:30,  1.07s/it]
37it [00:31,  1.03it/s]
38it [00:32,  1.13s/it]
39it [00:32,  1.15it/s]
40it [00:34,  1.21s/it]
41it [00:35,  1.11it/s]
43it [00:35,  1.38it/s]
44it [00:35,  1.85it/s]
45it 

{4: 25, 2: 18, 6: 29, 3: 34, 13: 2, 9: 14, 7: 17, 5: 24, 1: 12, 8: 20, 11: 7, 10: 7, 12: 6, 16: 4, 18: 2, 17: 2, 22: 1, 14: 1, 15: 1}


In [128]:
create_2_class_dataset_test(test,'test.tsv')


0it [00:00, ?it/s]
15it [00:00, 143.78it/s]
31it [00:00, 145.87it/s]
47it [00:00, 147.74it/s]
62it [00:00, 147.26it/s]
78it [00:00, 149.34it/s]
93it [00:00, 148.88it/s]
108it [00:00, 148.85it/s]
124it [00:00, 149.41it/s]
140it [00:00, 151.23it/s]
156it [00:01, 151.61it/s]
172it [00:01, 152.63it/s]
187it [00:01, 151.04it/s]
202it [00:01, 149.39it/s]
218it [00:01, 150.31it/s]
234it [00:01, 152.47it/s]
250it [00:01, 152.14it/s]
266it [00:01, 152.55it/s]
282it [00:01, 150.81it/s]
298it [00:01, 152.30it/s]
314it [00:02, 151.31it/s]
330it [00:02, 151.76it/s]
346it [00:02, 151.94it/s]
362it [00:02, 152.36it/s]
378it [00:02, 151.94it/s]
394it [00:02, 151.04it/s]
410it [00:02, 150.96it/s]
426it [00:02, 152.37it/s]
442it [00:02, 149.57it/s]
457it [00:03, 149.30it/s]
473it [00:03, 150.74it/s]
489it [00:03, 152.25it/s]
505it [00:03, 153.14it/s]
521it [00:03, 151.03it/s]
537it [00:03, 150.95it/s]
553it [00:03, 150.84it/s]
569it [00:03, 150.41it/s]
585it [00:03, 149.45it/s]
600it [00:03, 147.87it/s

In [129]:
create_2_class_dataset_test(dev,'dev-unscored.tsv')


0it [00:00, ?it/s]
16it [00:00, 154.97it/s]
32it [00:00, 155.82it/s]
48it [00:00, 154.51it/s]
64it [00:00, 153.96it/s]
80it [00:00, 153.22it/s]
96it [00:00, 154.44it/s]
112it [00:00, 155.32it/s]
128it [00:00, 153.45it/s]
143it [00:00, 151.79it/s]
158it [00:01, 150.96it/s]
173it [00:01, 150.37it/s]
188it [00:01, 147.03it/s]
203it [00:01, 143.79it/s]
218it [00:01, 145.47it/s]
234it [00:01, 147.73it/s]
249it [00:01, 148.33it/s]
264it [00:01, 148.58it/s]

In [132]:
l[1:]

['b', 'c']

In [25]:
create_n_class_multi_dataset(dev,"dev.tsv")

226it [02:37,  1.57it/s]

{4: 25, 2: 18, 6: 29, 3: 34, 13: 2, 9: 14, 7: 17, 5: 24, 1: 12, 8: 20, 11: 7, 10: 7, 12: 6, 16: 4, 18: 2, 17: 2, 22: 1, 14: 1, 15: 1}


In [22]:
create_n_class_multi_dataset(train[0:100],"train100.tsv")

8it [00:15,  1.62s/it]

KeyError: 'NEG'